In [9]:
import pandas as pd
import numpy as np
from pathlib import Path
from functions import (load_month, load_month_sa, load_month_pen, mapd_clean_merge)
!pip install openpyxl

# Settings
monthlist = [f"{m:02d}" for m in range(1, 3)]  # "01", "02"
y = 2015


[notice] A new release of pip is available: 24.2 -> 26.0.1
[notice] To update, run: pip install --upgrade pip


In [3]:
test = load_month("01", 2015)
print(test.shape)
print(test.head())

(2251623, 19)
  contractid  planid                                 org_type  \
0      90091     NaN                         HCPP - 1833 Cost   
1      E0654   801.0  Employer/Union Only Direct Contract PDP   
2      E0654   801.0  Employer/Union Only Direct Contract PDP   
3      E0654   801.0  Employer/Union Only Direct Contract PDP   
4      E0654   801.0  Employer/Union Only Direct Contract PDP   

                                 plan_type partd snp eghp  \
0                         HCPP - 1833 Cost    No  No   No   
1  Employer/Union Only Direct Contract PDP   Yes  No  Yes   
2  Employer/Union Only Direct Contract PDP   Yes  No  Yes   
3  Employer/Union Only Direct Contract PDP   Yes  No  Yes   
4  Employer/Union Only Direct Contract PDP   Yes  No  Yes   

                                           org_name  \
0  UNITED MINE WORKERS OF AMERICA HLTH & RETIREMENT   
1             IBT VOLUNTARY EMPLOYEE BENEFITS TRUST   
2             IBT VOLUNTARY EMPLOYEE BENEFITS TRUST   
3       

In [6]:
# Plan (enrollment & contract) data 
# Load and combine monthly data
plan_data = pd.concat([load_month(m, y) for m in monthlist], ignore_index=True)

# Sort data
plan_data = plan_data.sort_values(['contractid', 'planid', 'state', 'county', 'month'])

# Fill fips by state/county groups
plan_data['fips'] = plan_data.groupby(['state', 'county'])['fips'].transform(
    lambda x: x.ffill().bfill()
)

# Fill plan-level attributes
plan_cols = ['plan_type', 'partd', 'snp', 'eghp', 'plan_name']
for col in plan_cols:
    plan_data[col] = plan_data.groupby(['contractid', 'planid'])[col].transform(
        lambda x: x.ffill().bfill()
    )

# Fill contract-level attributes
contract_cols = ['org_type', 'org_name', 'org_marketing_name', 'parent_org']
for col in contract_cols:
    plan_data[col] = plan_data.groupby(['contractid'])[col].transform(
        lambda x: x.ffill().bfill()
    )

# Sort for aggregation
plan_data = plan_data.sort_values(['contractid', 'planid', 'fips', 'year', 'month'])

# Aggregate to yearly level
def agg_plan_year(group):
    nonmiss = group['enrollment'].notna()
    n = nonmiss.sum()
    enroll_vals = group.loc[nonmiss, 'enrollment']
    
    return pd.Series({
        'n_nonmiss': n,
        'avg_enrollment': enroll_vals.mean() if n > 0 else np.nan,
        'sd_enrollment': enroll_vals.std() if n > 1 else np.nan,
        'min_enrollment': enroll_vals.min() if n > 0 else np.nan,
        'max_enrollment': enroll_vals.max() if n > 0 else np.nan,
        'first_enrollment': enroll_vals.iloc[0] if n > 0 else np.nan,
        'last_enrollment': enroll_vals.iloc[-1] if n > 0 else np.nan,
        'state': group['state'].iloc[-1],
        'county': group['county'].iloc[-1],
        'org_type': group['org_type'].iloc[-1],
        'plan_type': group['plan_type'].iloc[-1],
        'partd': group['partd'].iloc[-1],
        'snp': group['snp'].iloc[-1],
        'eghp': group['eghp'].iloc[-1],
        'org_name': group['org_name'].iloc[-1],
        'org_marketing_name': group['org_marketing_name'].iloc[-1],
        'plan_name': group['plan_name'].iloc[-1],
        'parent_org': group['parent_org'].iloc[-1],
        'contract_date': group['contract_date'].iloc[-1]
    })

plan_data_2015 = plan_data.groupby(['contractid', 'planid', 'fips', 'year']).apply(agg_plan_year).reset_index()
print(f"Plan data shape: {plan_data_2015.shape}")

/tmp/ipykernel_1872428/1774076550.py:17: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  lambda x: x.ffill().bfill()
/tmp/ipykernel_1872428/1774076550.py:58: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  plan_data_2015 = plan_data.groupby(['contractid', 'planid', 'fips', 'year']).apply(agg_plan_year).reset_index()


Plan data shape: (2258499, 23)


In [8]:
folder = "../ma-data/ma/service-area/Extracted Data/"
import os
print(os.listdir(folder))

['MA_Cnty_SA_2011_07.csv', 'MA_Cnty_SA_2020_07.csv', 'MA_Cnty_SA_2015_06.csv', 'MA_Cnty_SA_2022_03.csv', 'MA_Cnty_SA_2023_02.csv', 'MA_Cnty_SA_2023_01.csv', 'MA_Cnty_SA_2022_01.csv', 'Read_Me_MA_Cnty_SA_2021.txt', 'MA_Cnty_SA_2015_09.csv', 'MA_Cnty_SA_2024_09.csv', 'MA_Cnty_SA_2023_06.csv', 'MA_Cnty_SA_2015_10.csv', 'MA_Cnty_SA_2009_06.csv', 'MA_Cnty_SA_2010_02.csv', 'MA_Cnty_SA_2018_10.csv', 'MA_Cnty_SA_2021_11.csv', 'MA_Cnty_SA_2020_09.csv', 'Read_Me_MA_Cnty_SA_2018.txt', 'MA_Cnty_SA_2013_02.csv', 'MA_Cnty_SA_200805.csv', 'MA_Cnty_SA_2018_07.csv', 'MA_Cnty_SA_2018_09.csv', 'MA_Cnty_SA_2021_09.csv', 'MA_Cnty_SA_2020_08.csv', 'MA_Cnty_SA_2020_10.csv', 'Read_Me_MA_Cnty_SA_2023.txt', 'Read_Me_MA_Cnty_SA_2024.txt', 'MA_Cnty_SA_2016_01.csv', 'MA_Cnty_SA_2009_12.csv', 'MA_Cnty_SA_2011_10.csv', 'MA_Cnty_SA_2016_07.csv', 'MA_Cnty_SA_2013_07.csv', 'MA_Cnty_SA_2024_08.csv', 'MA_Cnty_SA_2017_02.csv', 'MA_Cnty_SA_2014_02.csv', 'MA_Cnty_SA_2019_03.csv', 'MA_Cnty_SA_2024_07.csv', 'MA_Cnty_SA_2020_0

In [5]:
import os
os.getcwd()

'/home/pturn22/econ470/a0/work/hwk2'

In [10]:
import os

sa_folder = "../ma-data/ma/service-area/Extracted Data"
files = [f for f in os.listdir(sa_folder) if f.startswith(f"MA_Cnty_SA_{y}_") and f.endswith(".csv")]
files = [os.path.join(sa_folder, f) for f in files]  # full paths
print("Found service area files:")
print(files)

Found service area files:
['../ma-data/ma/service-area/Extracted Data/MA_Cnty_SA_2015_06.csv', '../ma-data/ma/service-area/Extracted Data/MA_Cnty_SA_2015_09.csv', '../ma-data/ma/service-area/Extracted Data/MA_Cnty_SA_2015_10.csv', '../ma-data/ma/service-area/Extracted Data/MA_Cnty_SA_2015_04.csv', '../ma-data/ma/service-area/Extracted Data/MA_Cnty_SA_2015_08.csv', '../ma-data/ma/service-area/Extracted Data/MA_Cnty_SA_2015_05.csv', '../ma-data/ma/service-area/Extracted Data/MA_Cnty_SA_2015_02.csv', '../ma-data/ma/service-area/Extracted Data/MA_Cnty_SA_2015_03.csv', '../ma-data/ma/service-area/Extracted Data/MA_Cnty_SA_2015_11.csv', '../ma-data/ma/service-area/Extracted Data/MA_Cnty_SA_2015_12.csv', '../ma-data/ma/service-area/Extracted Data/MA_Cnty_SA_2015_01.csv', '../ma-data/ma/service-area/Extracted Data/MA_Cnty_SA_2015_07.csv']


In [12]:
service_year = pd.concat(
    [
        pd.read_csv(f"../ma-data/ma/service-area/Extracted Data/MA_Cnty_SA_{y}_{m}.csv")
          .rename(columns=lambda x: x.strip().lower())  # lowercase & strip spaces
        for m in monthlist
    ],
    ignore_index=True
)

print(service_year.columns.tolist())  # check if 'contractid' is there

/tmp/ipykernel_1872428/128791298.py:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv(f"../ma-data/ma/service-area/Extracted Data/MA_Cnty_SA_{y}_{m}.csv")
/tmp/ipykernel_1872428/128791298.py:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv(f"../ma-data/ma/service-area/Extracted Data/MA_Cnty_SA_{y}_{m}.csv")
/tmp/ipykernel_1872428/128791298.py:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv(f"../ma-data/ma/service-area/Extracted Data/MA_Cnty_SA_{y}_{m}.csv")
/tmp/ipykernel_1872428/128791298.py:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv(f"../ma-data/ma/service-area/Extracted Data/MA_Cnty_SA_{y}_{m}.csv")
/tmp/ipykernel_1872428/128791298.py:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on 

['contract id', 'organization name', 'organization type', 'plan type', 'partial', 'eghp', 'ssa', 'fips', 'county', 'state', 'notes']


In [13]:
service_year = pd.concat(
    [
        pd.read_csv(f"../ma-data/ma/service-area/Extracted Data/MA_Cnty_SA_{y}_{m}.csv")
          .rename(columns=lambda x: x.strip().lower())
          .rename(columns={
              'contract id': 'contractid',
              'organization name': 'org_name',
              'organization type': 'org_type'
          })
        for m in monthlist
    ],
    ignore_index=True
)

print(service_year.columns.tolist())

/tmp/ipykernel_1872428/1544421988.py:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv(f"../ma-data/ma/service-area/Extracted Data/MA_Cnty_SA_{y}_{m}.csv")
/tmp/ipykernel_1872428/1544421988.py:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv(f"../ma-data/ma/service-area/Extracted Data/MA_Cnty_SA_{y}_{m}.csv")
/tmp/ipykernel_1872428/1544421988.py:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv(f"../ma-data/ma/service-area/Extracted Data/MA_Cnty_SA_{y}_{m}.csv")
/tmp/ipykernel_1872428/1544421988.py:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv(f"../ma-data/ma/service-area/Extracted Data/MA_Cnty_SA_{y}_{m}.csv")
/tmp/ipykernel_1872428/1544421988.py:3: DtypeWarning: Columns (10) have mixed types. Specify dtype optio

['contractid', 'org_name', 'org_type', 'plan type', 'partial', 'eghp', 'ssa', 'fips', 'county', 'state', 'notes']


In [20]:
service_year = service_year.rename(columns={'plan type': 'plan_type'})
# Load all monthly service area CSVs
service_year = pd.concat([
    pd.read_csv(f"../ma-data/ma/service-area/Extracted Data/MA_Cnty_SA_{y}_{m}.csv",
                dtype=str, low_memory=False)
    for m in monthlist
], ignore_index=True)

# Standardize column names: lowercase + remove spaces
service_year.columns = service_year.columns.str.strip().str.lower().str.replace(' ', '_')

# Now let's check the columns
print(service_year.columns.tolist())

['contract_id', 'organization_name', 'organization_type', 'plan_type', 'partial', 'eghp', 'ssa', 'fips', 'county', 'state', 'notes']


In [24]:
# After reading CSVs into service_year
service_year.columns = service_year.columns.str.strip()  # remove leading/trailing spaces
service_year = service_year.rename(columns={
    'contract id': 'contract_id',
    'organization name': 'organization_name',
    'organization type': 'organization_type',
    'plan type': 'plan_type'
})

In [26]:
# Service Data
# Set working directory (optional, just to be sure)
os.getcwd()

# Year and months
y = 2015
monthlist = [f"{i:02d}" for i in range(1, 13)]  # '01' to '12'

# Function to clean column names for consistency
def clean_service_columns(df):
    # Strip spaces, lowercase, replace spaces with underscores
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
    
    # Rename known columns to standard names
    df = df.rename(columns={
        'contractid': 'contract_id',
        'contract_id': 'contract_id',
        'organization_name': 'org_name',
        'organization_type': 'org_type',
        'plan_type': 'plan_type'
    })
    return df

# Load all monthly service area CSVs
service_dfs = []
for m in monthlist:
    path = f"../ma-data/ma/service-area/Extracted Data/MA_Cnty_SA_{y}_{m}.csv"
    if os.path.exists(path):
        df = pd.read_csv(path, dtype=str, encoding="latin-1")
        df = clean_service_columns(df)
        df['month'] = int(m)
        df['year'] = y
        service_dfs.append(df)
    else:
        print(f"Warning: file not found: {path}")

# Combine all months
service_year = pd.concat(service_dfs, ignore_index=True)

# Check columns
print("Columns after cleaning:", list(service_year.columns))

# Sort for stable fills
service_year = service_year.sort_values(['contract_id', 'fips', 'state', 'county', 'month'])

# Fill fips by state/county groups
service_year['fips'] = service_year.groupby(['state', 'county'])['fips'].transform(
    lambda x: x.ffill().bfill()
)

# Fill contract-level attributes
contract_cols_sa = ['plan_type', 'partial', 'eghp', 'org_type', 'org_name']
for col in contract_cols_sa:
    if col in service_year.columns:
        service_year[col] = service_year.groupby(['contract_id'])[col].transform(
            lambda x: x.ffill().bfill()
        )

# Collapse to yearly: one row per contract x county (fips) x year
service_year = service_year.sort_values(['contract_id', 'fips', 'year', 'month'])

service_data_2015 = service_year.groupby(['contract_id', 'fips', 'year']).agg({
    'state': 'last',
    'county': 'last',
    'org_name': 'last',
    'org_type': 'last',
    'plan_type': 'last',
    'partial': 'last',
    'eghp': 'last',
    'ssa': 'last',
    'notes': 'last'
}).reset_index()

print(f"Service area data shape: {service_data_2015.shape}")

Columns after cleaning: ['contract_id', 'org_name', 'org_type', 'plan_type', 'partial', 'eghp', 'ssa', 'fips', 'county', 'state', 'notes', 'month', 'year']


/tmp/ipykernel_1872428/1787193820.py:56: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  lambda x: x.ffill().bfill()
/tmp/ipykernel_1872428/1787193820.py:56: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  lambda x: x.ffill().bfill()


Service area data shape: (377723, 12)


In [27]:
# Plan Characteristics (premium) Data

y = 2015

# --- MA landscape data (A to M) ---
ma_path_a = "../ma-data/ma/landscape/Extracted Data/2015LandscapeSource file MA_AtoM 11042014.csv"
ma_col_names = ["state", "county", "org_name", "plan_name", "plan_type", "premium", "partd_deductible",
                "drug_type", "gap_coverage", "drug_type_detail", "contractid",
                "planid", "segmentid", "moop", "star_rating"]

ma_data_a = pd.read_csv(ma_path_a, skiprows=6, dtype=str, encoding="latin-1")
ma_data_a.columns = ma_col_names[:len(ma_data_a.columns)]

# Clean numeric columns
for col in ['premium', 'partd_deductible']:
    ma_data_a[col] = ma_data_a[col].str.replace('-', '0')
    ma_data_a[col] = pd.to_numeric(ma_data_a[col].str.replace(r'[^\d.]', '', regex=True), errors='coerce')

ma_data_a['planid'] = pd.to_numeric(ma_data_a['planid'], errors='coerce')
ma_data_a['segmentid'] = pd.to_numeric(ma_data_a['segmentid'], errors='coerce')


# --- MA landscape data (N to W) ---
ma_path_b = "../ma-data/ma/landscape/Extracted Data/2015LandscapeSource file MA_NtoW 11042014.csv"
ma_data_b = pd.read_csv(ma_path_b, skiprows=6, dtype=str, encoding="latin-1")
ma_data_b.columns = ma_col_names[:len(ma_data_b.columns)]

# Clean numeric columns
for col in ['premium', 'partd_deductible']:
    ma_data_b[col] = ma_data_b[col].str.replace('-', '0')
    ma_data_b[col] = pd.to_numeric(ma_data_b[col].str.replace(r'[^\d.]', '', regex=True), errors='coerce')

ma_data_b['planid'] = pd.to_numeric(ma_data_b['planid'], errors='coerce')
ma_data_b['segmentid'] = pd.to_numeric(ma_data_b['segmentid'], errors='coerce')

# Combine A-M and N-W
ma_data = pd.concat([ma_data_a, ma_data_b], ignore_index=True)


# ================================
# MA-PD landscape data (Part D)
# ================================

mapd_path = "../ma-data/ma/landscape/Extracted Data/PartCD/2015/Medicare Part D 2015 Plan Report 03182015.xls"
mapd_col_names = ["state", "county", "org_name", "plan_name", "contractid", "planid", "segmentid",
                  "org_type", "plan_type", "snp", "snp_type", "benefit_type", "below_benchmark",
                  "national_pdp", "premium_partc",
                  "premium_partd_basic", "premium_partd_supp", "premium_partd_total",
                  "partd_assist_full", "partd_assist_75", "partd_assist_50", "partd_assist_25",
                  "partd_deductible", "deductible_exclusions", "increase_coverage_limit",
                  "gap_coverage", "gap_coverage_type"]

# Read Excel sheets without forcing column names
mapd_data_a = pd.read_excel(mapd_path, sheet_name="Alabama to Montana", skiprows=4, nrows=15854)
mapd_data_b = pd.read_excel(mapd_path, sheet_name="Nebraska to Wyoming", skiprows=4, nrows=20300)

# Rename columns to match your expected list (truncate if needed)
mapd_data_a.columns = mapd_col_names[:len(mapd_data_a.columns)]
mapd_data_b.columns = mapd_col_names[:len(mapd_data_b.columns)]

# Combine sheets
mapd_data = pd.concat([mapd_data_a, mapd_data_b], ignore_index=True)


# ================================
# Clean and merge landscape + Part D
# ================================

landscape_2015 = mapd_clean_merge(ma_data, mapd_data, y)
print(f"Landscape data shape: {landscape_2015.shape}")

Landscape data shape: (45059, 11)


In [29]:
sample_file = f"{pen_base_path}/State_County_Penetration_MA_{y}_{monthlist[0]}.csv"
df_sample = pd.read_csv(sample_file, encoding="latin-1")
print(df_sample.columns.tolist())

['State Name', 'County Name', 'FIPSST', 'FIPSCNTY', 'FIPS', 'SSAST', 'SSACNTY', 'SSA', 'Eligibles', 'Enrolled', 'Penetration']


In [32]:
# Penetration Data
# Load and combine monthly penetration data
pen_base_path = "../ma-data/ma/penetration/Extracted Data"

def load_month_pen(m, y, base_path):
    path = f"{base_path}/State_County_Penetration_MA_{y}_{m}.csv"
    df = pd.read_csv(path, encoding="latin-1")
    
    # Rename columns
    df = df.rename(columns={
        'State Name': 'state',
        'County Name': 'county',
        'FIPS': 'fips',
        'SSA': 'ssa',
        'Eligibles': 'eligibles',
        'Enrolled': 'enrolled'
    })
    
    # Remove commas and convert to numeric
    df['eligibles'] = pd.to_numeric(df['eligibles'].astype(str).str.replace(',', ''), errors='coerce')
    df['enrolled'] = pd.to_numeric(df['enrolled'].astype(str).str.replace(',', ''), errors='coerce')
    
    df["month"] = int(m)
    df["year"] = y
    return df

# Load all months
ma_penetration = pd.concat(
    [load_month_pen(m, y, pen_base_path) for m in monthlist],
    ignore_index=True
)

# Fill missing FIPS
ma_penetration['fips'] = ma_penetration.groupby(['state', 'county'])['fips'].transform(
    lambda x: x.ffill().bfill()
)

# Collapse to yearly
def agg_penetration(group):
    n_elig = group['eligibles'].notna().sum()
    n_enrol = group['enrolled'].notna().sum()
    elig_vals = group['eligibles'].dropna()
    enrol_vals = group['enrolled'].dropna()
    
    return pd.Series({
        'n_elig': n_elig,
        'n_enrol': n_enrol,
        'avg_eligibles': elig_vals.mean() if n_elig > 0 else np.nan,
        'sd_eligibles': elig_vals.std() if n_elig > 1 else np.nan,
        'min_eligibles': elig_vals.min() if n_elig > 0 else np.nan,
        'max_eligibles': elig_vals.max() if n_elig > 0 else np.nan,
        'first_eligibles': elig_vals.iloc[0] if n_elig > 0 else np.nan,
        'last_eligibles': elig_vals.iloc[-1] if n_elig > 0 else np.nan,
        'avg_enrolled': enrol_vals.mean() if n_enrol > 0 else np.nan,
        'sd_enrolled': enrol_vals.std() if n_enrol > 1 else np.nan,
        'min_enrolled': enrol_vals.min() if n_enrol > 0 else np.nan,
        'max_enrolled': enrol_vals.max() if n_enrol > 0 else np.nan,
        'first_enrolled': enrol_vals.iloc[0] if n_enrol > 0 else np.nan,
        'last_enrolled': enrol_vals.iloc[-1] if n_enrol > 0 else np.nan,
        'ssa': group['ssa'].iloc[-1] if 'ssa' in group.columns else np.nan
    })

pen_2015 = ma_penetration.groupby(['fips', 'state', 'county', 'year']).apply(agg_penetration).reset_index()
print(f"Penetration data shape: {pen_2015.shape}")

Penetration data shape: (3224, 19)


/tmp/ipykernel_1872428/1176406888.py:63: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  pen_2015 = ma_penetration.groupby(['fips', 'state', 'county', 'year']).apply(agg_penetration).reset_index()


In [35]:
import os
os.listdir("../ma-data/ma/cms-payment/2015")

['2015PartDPlans.xlsx',
 'Plan Payment Data Elements.pdf',
 '2015PartDReconciliation.xlsx',
 '2015PartCPlanLevel.xlsx',
 '2015PartCCountyLevel.xlsx']

In [33]:
import os
os.getcwd()

'/home/pturn22/econ470/a0/work/hwk2'

In [36]:
# Rebate Data
# Part C rebate data
ma_path_a = "../ma-data/ma/cms-payment/2015/2015PartCPlanLevel.xlsx"
risk_rebate_a = pd.read_excel(ma_path_a, skiprows=3, nrows=2824,
                               names=["contractid", "planid", "contract_name", "plan_type",
                                      "riskscore_partc", "payment_partc", "rebate_partc"])

# Clean Part C numeric columns
for col in ['riskscore_partc', 'payment_partc', 'rebate_partc']:
    risk_rebate_a[col] = pd.to_numeric(
        risk_rebate_a[col].astype(str).str.replace(r'[^\d.-]', '', regex=True),
        errors='coerce'
    )

risk_rebate_a['planid'] = pd.to_numeric(risk_rebate_a['planid'], errors='coerce')
risk_rebate_a['year'] = 2015

# Part D rebate data
ma_path_b = "../ma-data/ma/cms-payment/2015/2015PartDPlans.xlsx"
risk_rebate_b = pd.read_excel(ma_path_b, skiprows=3, nrows=3898,
                               names=["contractid", "planid", "contract_name", "plan_type",
                                      "directsubsidy_partd", "riskscore_partd", "reinsurance_partd",
                                      "costsharing_partd"])

# Clean Part D numeric columns
for col in ['directsubsidy_partd', 'reinsurance_partd', 'costsharing_partd']:
    risk_rebate_b[col] = pd.to_numeric(
        risk_rebate_b[col].astype(str).str.replace(r'[^\d.-]', '', regex=True),
        errors='coerce'
    )

# Total Part D payment
risk_rebate_b['payment_partd'] = (risk_rebate_b['directsubsidy_partd'] + 
                                   risk_rebate_b['reinsurance_partd'] + 
                                   risk_rebate_b['costsharing_partd'])

risk_rebate_b['planid'] = pd.to_numeric(risk_rebate_b['planid'], errors='coerce')

# Select relevant columns for Part D
risk_rebate_b = risk_rebate_b[['contractid', 'planid', 'payment_partd',
                               'directsubsidy_partd', 'reinsurance_partd', 'costsharing_partd',
                               'riskscore_partd']]

# Merge Part C and Part D data
rebate_2015 = risk_rebate_a.merge(risk_rebate_b, on=['contractid', 'planid'], how='left')

print(f"Rebate data shape: {rebate_2015.shape}")

Rebate data shape: (2741, 13)


In [40]:
import os
# List files in the directory where the CSV should be
ffs_dir = "../ma-data/ffs-costs/Extracted Data/Aged Only"
print(os.listdir(ffs_dir))

['Aged07.csv', 'FFS18.xlsx', 'AGED08.csv', 'FFS16.xlsx', 'aged13.csv', 'aged10.csv', 'aged09.csv', 'aged11.csv', 'aged14.csv', 'aged12.csv', 'FFS15.xlsx', 'AGED06.csv', 'FFS17_AdvancedNotice.xlsx']


In [42]:
# FFS costs data
ffs_path = "../ma-data/ffs-costs/Extracted Data/Aged Only/FFS15.xlsx"
ffs_col_names = ["ssa", "state", "county_name", "parta_enroll",
                 "parta_reimb", "parta_percap", "parta_reimb_unadj",
                 "parta_percap_unadj", "parta_ime", "parta_dsh",
                 "parta_gme", "partb_enroll",
                 "partb_reimb", "partb_percap",
                 "mean_risk"]

ffs_data = pd.read_excel(ffs_path, skiprows=2, names=ffs_col_names, na_values=['*', '.'],
                       usecols=range(15))

# Select and clean
ffscosts_2015 = ffs_data[['ssa', 'state', 'county_name', 'parta_enroll', 'parta_reimb',
                          'partb_enroll', 'partb_reimb', 'mean_risk']].copy()
ffscosts_2015['year'] = 2015
ffscosts_2015['ssa'] = pd.to_numeric(ffscosts_2015['ssa'], errors='coerce')

for col in ['parta_enroll', 'parta_reimb', 'partb_enroll', 'partb_reimb', 'mean_risk']:
    ffscosts_2015[col] = pd.to_numeric(ffscosts_2015[col].astype(str).str.replace(r'[^\d.-]', '', regex=True), errors='coerce')

print(f"FFS costs data shape: {ffscosts_2015.shape}")

FFS costs data shape: (3224, 9)


In [47]:
service_data_2015.columns.tolist()

['contract_id',
 'fips',
 'year',
 'state',
 'county',
 'org_name',
 'org_type',
 'plan_type',
 'partial',
 'eghp',
 'ssa',
 'notes']

In [48]:
# Merge Data

# Ensure fips is string for merging
plan_data_2015['fips'] = plan_data_2015['fips'].astype(str)
service_data_2015['fips'] = service_data_2015['fips'].fillna(0).astype(int).astype(str)

# Merge plan data with service area
ma_2015 = plan_data_2015.merge(
    service_data_2015[['contract_id', 'fips']],
    left_on=['contractid', 'fips'],
    right_on=['contract_id', 'fips'],
    how='inner'
)

# Filter out territories and certain plan types
excluded_states = ['VI', 'PR', 'MP', 'GU', 'AS', '']
ma_2015 = ma_2015[
    (~ma_2015['state'].isin(excluded_states)) &
    (ma_2015['snp'] == 'No') &
    ((ma_2015['planid'] < 800) | (ma_2015['planid'] >= 900)) &
    (ma_2015['planid'].notna()) &
    (ma_2015['fips'].notna())
]

# Ensure fips is string for penetration merge
pen_2015['fips'] = pen_2015['fips'].astype(str)

# Prepare penetration data for join
pen_2015_join = pen_2015.copy()
pen_2015_join = pen_2015_join.rename(columns={'state': 'state_long', 'county': 'county_long'})
pen_2015_join['state_long'] = pen_2015_join['state_long'].str.lower()

# Keep only unique fips entries
pen_2015_join['ncount'] = pen_2015_join.groupby('fips')['fips'].transform('count')
pen_2015_join = pen_2015_join[pen_2015_join['ncount'] == 1].drop(columns=['ncount'])

# Join penetration data
ma_2015 = ma_2015.merge(pen_2015_join, on='fips', how='left', suffixes=('', '_pen'))

# Create state name lookup safely
state_2015 = ma_2015.groupby('state').agg(
    state_name=('state_long', lambda x: x.dropna().iloc[-1] if len(x.dropna()) > 0 else None)
).reset_index()

# Join state names
full_2015 = ma_2015.merge(state_2015, on='state', how='left')

# Prepare landscape data for join
landscape_2015_join = landscape_2015.copy()
landscape_2015_join['state'] = landscape_2015_join['state'].str.lower()

# Join landscape data
full_2015 = full_2015.merge(
    landscape_2015_join,
    left_on=['contractid', 'planid', 'state_name', 'county'],
    right_on=['contractid', 'planid', 'state', 'county'],
    how='left',
    suffixes=('', '_land')
)

# Join rebate data (exclude contract_name and plan_type from rebate)
rebate_2015_join = rebate_2015.drop(columns=['contract_name', 'plan_type'], errors='ignore')
full_2015 = full_2015.merge(rebate_2015_join, on=['contractid', 'planid'], how='left', suffixes=('', '_reb'))

# Calculate basic_premium
def calc_basic_premium(row):
    if row.get('rebate_partc', 0) > 0:
        return 0
    elif row.get('partd') == 'No' and pd.notna(row.get('premium')) and pd.isna(row.get('premium_partc')):
        return row.get('premium')
    else:
        return row.get('premium_partc')

# Calculate bid
def calc_bid(row):
    rebate = row.get('rebate_partc', 0) or 0
    basic_premium = row.get('basic_premium', 0) or 0
    payment_partc = row.get('payment_partc')
    riskscore_partc = row.get('riskscore_partc')
    
    if pd.isna(payment_partc) or pd.isna(riskscore_partc) or riskscore_partc == 0:
        return np.nan
    elif rebate == 0 and basic_premium > 0:
        return (payment_partc + basic_premium) / riskscore_partc
    elif rebate > 0 or basic_premium == 0:
        return payment_partc / riskscore_partc
    else:
        return np.nan

full_2015['basic_premium'] = full_2015.apply(calc_basic_premium, axis=1)
full_2015['bid'] = full_2015.apply(calc_bid, axis=1)

print(f"Final merged data shape: {full_2015.shape}")

Final merged data shape: (51154, 62)


In [49]:
# Save to CSV
output_path = "../data/output/data-2015.csv"
Path(output_path).parent.mkdir(parents=True, exist_ok=True)
full_2015.to_csv(output_path, index=False)
print(f"Data saved to {output_path}")

Data saved to ../data/output/data-2015.csv
